# Loading data

In [ ]:
import numpy as np
import osyris

## Loading a Ramses output

We load a data output from a star formation simulation

In [ ]:
path = "osyrisdata/starformation"
data = osyris.Dataset(8, scale="au", path=path).load()

and show a 2D histogram of the gas density versus the magnetic field strength

In [ ]:
osyris.plot.histogram(data["hydro"]["density"], data["hydro"]["B_field"],
                      norm="log", loglog=True)

## Selective loading

### Filtering on cell values

It is possible to load only a subset of the cells, by using custom functions to perform the selection.

As an example, to load all the cells with $\rho > 10^{-15}~{\rm g~cm}^{-3}$, we use a selection criterion

In [ ]:
data = osyris.Dataset(8, scale="au", path=path).load(
    select={"hydro": {"density": lambda d : d > 1.0e-15 * osyris.units('g/cm**3')}})

and we can now see in the resulting histogram that all the low-density cells have been left out:

In [ ]:
osyris.plot.histogram(data["hydro"]["density"], data["hydro"]["B_field"],
                      norm="log", loglog=True)

Multiple selection criteria are ANDed:

In [ ]:
data = osyris.Dataset(8, scale="au", path=path).load(
    select={"hydro": {"density": lambda d : d > 1.0e-16 * osyris.units('g/cm**3')},
            "amr": {"xyz_x": lambda x : x > 1500. * osyris.units('au')}})

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             dx=1000 * osyris.units('au'),
             origin=data["amr"]["xyz"][np.argmax(data["hydro"]["density"]).values],
             direction='z')

### Only loading certain variables

It is also possible to skip a variable entirely by setting the key to `False` in the `select` dict:

In [ ]:
data = osyris.Dataset(8, scale="au", path=path).load(
    select={"hydro": {"density": False}})
"density" in data["hydro"]

Entire groups of variables can also be skipped.
The different groups are `'amr'`, `'hydro'`, `'grav'` (or `'gravity'`), and `'rt'`,
but note that the `'amr'` group is always loaded.
Loading all but the hydro variables can be achieved via

In [ ]:
data = osyris.Dataset(8, scale="au", path=path).load(
    select={"hydro": False})
data

### Selecting AMR levels

Selecting AMR levels uses the same syntax as selecting other variables,
but works slightly differently under the hood.
A maximum level will be found by testing the selection function provided,
and only the required levels will be traversed by the loader,
thus speeding up the loading process.

Hence,

In [ ]:
data = osyris.Dataset(8, scale="au", path=path).load(
    select={"amr": {"level": lambda l : l < 7}})
data["amr"]["level"]

will only read levels 1 to 6, while

In [ ]:
data = osyris.Dataset(8, scale="au", path=path).load(
    select={"amr": {"level": lambda l : np.logical_and(l > 5, l < 9)}})
data["amr"]["level"]

will read levels 1 to 8, but will then discard all cells with `level` < 6.

### Loading only selected CPU outputs

It is also possible to feed a list of CPU numbers to read from.

In [ ]:
data = osyris.Dataset(8, scale="au", path=path).load(
    cpu_list=[1, 2, 10, 4, 5])

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             dx=2000 * osyris.units("au"),
             origin=data["amr"]["xyz"][np.argmax(data["hydro"]["density"]).values],
             direction='z')

<div class="alert alert-info">

**Note**

When performing a selection (using `select`) on spatial position `x`, `y`, or `z`,
and if the ordering of the cells in the AMR mesh is using the Hilbert curve,
then a pre-selection is automatically made on the CPU files to load to speed-up the loading.

</div>